## Script to download Sentinel 1 SAR data

In [ ]:
# importing necessary modules
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

# to access data using api, provide the cropernicus login details
api = SentinelAPI('#username', '#password','https://apihub.copernicus.eu/apihub')

# download data with ROI
footprint = geojson_to_wkt(read_geojson('studyarea.geojson'))

# retreive all products from API with filters 
products = api.query(footprint,
                     date=('20180801', '20180825'),
                     producttype='GRD',
                     platformname='Sentinel-1',
                     relativeorbitnumber = 168,
                     slicenumber = 21,
                    filename = 'S1A*',
                     orbitdirection = "DESCENDING"
                     
                    )

# storing JSON in dataframe for better understanding 
products_df = api.to_dataframe(products)

# no of products
print(len(products_df))

# to calculate the average storage in GB it occupies
# print(len(products_df)*4)

# download the products
api.download_all(products)

## Function to process SAR data and retreive intensity and phase values at paticular x,y

Note:
Currently, Snappy (python API for SNAP) offers this processing via function calls but requires SNAP software to be installed and limited to specific python versions (2.7,3.4,3.5,3.6). I have used python version 3.6 to perform geocoding. 

### Initial processing till terrain correction

In [ ]:
# importing necessary modules

from snappy import jpy, ProgressMonitor, ProductIO
from snappy import HashMap

import os, gc
from snappy import GPF

In [170]:
def terrain_corrected(path,filename,polarization,subswath,burstStart,burstEnd):
    
    file_path = path
    pol = polarization
    sub_swath = subswath
    b_start = burstStart
    b_end = burstEnd
    filename = filename
    
    source_bands_needed = f"Intensity_{subswath}_{polarization},i_{subswath}_{polarization},q_{subswath}_{polarization}"

    params={
        # TOPSAR-split
        "subswath": sub_swath,
        "selectedPolarisations": pol,
        "firstBurstIndex": b_start,
        "lastBurstIndex": b_end,

        # Apply-Orbit-File
        "continuteOnFail": True,
        "orbitType" : "Sentinel Precise (Auto Download)",
        # TOPSAR-Deburst

        # Terrain-Correction
        "demName":'SRTM 1Sec HGT',
        "pixelSpacingInMeter": 10.0,
        "sourceBands": source_bands_needed,
        "outputComplex": True

    }
    
#     reading the sar data from path (make sure it is .zip file path)
    prod_data = ProductIO.readProduct(path)
    
    GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

#     topsarsplit
    t1_spt = GPF.createProduct("TOPSAR-Split", parameters, prod_data)
    # ProductIO.writeProduct(t1_spt, "splt", 'BEAM-DIMAP')

#     Applyorbitfile
    t1_spt_orb = GPF.createProduct("Apply-Orbit-File", parameters, t1_spt)
    # ProductIO.writeProduct(t1_spt_orb, "splt_orb", 'BEAM-DIMAP')

#     deburst
    dbt = GPF.createProduct("TOPSAR-Deburst", parameters, t1_spt_orb)
    # ProductIO.writeProduct(dbt, "splt_orb_db", 'BEAM-DIMAP')

#     range-doppler terrain correction
    TC = GPF.createProduct("Terrain-Correction", parameters, dbt)
    # ProductIO.writeProduct(TC, "splt_orb_db_TC3", 'BEAM-DIMAP')
    
#     saving geocoded product in GEOTIFF format
    ProductIO.writeProduct(TC, filename+"splt_orb_db_TC4", 'GEOTIFF')
    
    return
    
    

In [172]:
# user declared variables
base_path = r'C:\Users\ge26fax\Documents\Remote_Sensing_Seminar\Project\slaves'
directory = '20170203'
filename = 'S1A_IW_SLC__1SDV_20170203T052728_20170203T052755_015115_018B6D_FA11.zip'

# Combine the variables to create the complete file path
file_path = rf"{base_path}\{directory}\{filename}"

# according to user specifications
polarization = 'VV'
subswath = 'IW3'
burstStart = 6
burstEnd = 6



In [ ]:
# call the function to get geocoded product
terrain_corrected(path,filename, polarization, subswath, burstStart, burstEnd)

### Reading geocoded product in rasterio and saving the raw intensity and complex values

Note: Rasterio has to be installed before using it. It has GDAL as dependency, so make sure that GDAL is also installed.

In [ ]:
# importing rasterio library 
import rasterio
import datetime
import pandas as pd

# Define an empty dictionary with the column names
data = {
    'time': [],
    'intensity': [],
    'complex_i': [],
    'complex_q_phase': []
}

# Create a DataFrame from the empty dictionary
df = pd.DataFrame(data)

# function to calculate time
def know_the_unix_time(filename):
    # Define the input filename as a string
    filename = filename

    # Extract the datetime part of the filename
    datetime_str = filename.split('_')[5]

    # Convert the datetime string to a datetime object
    datetime_obj = datetime.datetime.strptime(datetime_str, '%Y%m%dT%H%M%S')

    #  Epoch time
    epoch_time = datetime_obj.timestamp()
    # Print the epoch time
    return epoch_time


In [123]:

# function to get the pixel values
def get_intensity_phase_values(path,filename,x,y,df):

    # give the path to the geocoded product
    dataset = rasterio.open(path)
    
    # get the time
    epoch_time = know_the_unix_time(filename)
    
    # getting values of intensity,complex part     
    val =list(dataset.sample([(x,y)]))
    
    # the bands are ordered as intensity,complex i and complex q
    # intensity
    intensity_value = val[0][0]
    # complex_i
    complex_i_value = val[0][1]
    # complex_q (phase)
    complex_q_value = val[0][2]
    
    # appending the datarecord to the df
    
    df.loc[len(df.index)] = [epoch_time, intensity_value,complex_i_value,complex_q_value]

    return df

In [ ]:
# sample x and y values

x = (dataset.bounds.left + dataset.bounds.right) / 2.0
y = (dataset.bounds.bottom + dataset.bounds.top) / 2.0

# path to the geocoded tif file from the terrain_corrected function
path  = r'C:/Users/ge26fax/Documents/Remote_Sensing_Seminar/S1A_IW_SLC__1SDV_20170203T052728_20170203T052755_015115_018B6D_FA11.zipsplt_orb_db_TC4.tif'
filename = 'S1A_IW_SLC__1SDV_20170203T052728_20170203T052755_015115_018B6D_FA11.zip'


In [ ]:
# get_intensity_phase_values function call

get_intensity_phase_values(path,filename,x,y,df)

Authors: Jayendra Praveen Kumar Chorapalli
Affiliation: M.Sc TUM 
Program: ESPACE
contact: jayendra.chorapalli@tum.de